In [1]:
from preprocessing import Preprocessing
import tfidf
from sklearn.cluster import AffinityPropagation
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics import silhouette_score

In [2]:
prp = Preprocessing()
df = prp.main()
df = Preprocessing.preprocess(df)
df = df['text']
ntf = tfidf.TFIDF(df)
X = ntf.df_tfidf_vectorize(df)

starting preprocessing...
...preprocessing completed
starting vectorizing words...
...words vectorized


In [3]:
vectoriser = TfidfVectorizer(ngram_range=(1,2), max_features=100)
vectoriser.fit_transform(df)
afprop = AffinityPropagation(max_iter=400)
afprop.fit(X)

/home/annabelle/Documents/TwiiterSentimentAnalysis/venv/lib/python3.8/site-packages/sklearn/cluster/_affinity_propagation.py:250: ConvergenceWarning: Affinity propagation did not converge, this model will not have any cluster centers.
  warnings.warn(


AffinityPropagation(max_iter=400)

In [4]:
result = pd.concat([df,pd.DataFrame(X.toarray(),columns=vectoriser.get_feature_names())],axis=1)
result['cluster'] = afprop.predict(X)

/home/annabelle/Documents/TwiiterSentimentAnalysis/venv/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/home/annabelle/Documents/TwiiterSentimentAnalysis/venv/lib/python3.8/site-packages/sklearn/cluster/_affinity_propagation.py:528: ConvergenceWarning: This model does not have any cluster centers because affinity propagation did not converge. Labeling every sample as '-1'.
  warnings.warn(


In [5]:
result[['text', 'cluster']]

,text,cluster
0,nighti night this twitter thing is confus but ...,-1
1,he he im good thanx ola cant wait until tomorr...,-1
2,i wanna talk to you,-1
3,im look forward to queen tomorrow too andi mur...,-1
4,ew realli that full of fail,-1
...,...,...
13995,gotta wake my bop up outta this lame ass tranc...,-1
13996,why are you ok i am ill and i feel like i am g...,-1
13997,you make evryth better i miss being your bestf...,-1
13998,mom i hope your day was fill with love amp hap...,-1


In [6]:
cluster_centers_indices = afprop.cluster_centers_indices_
labels = afprop.labels_
labels

array([-1, -1, -1, ..., -1, -1, -1])

In [7]:
n_clusters_ = len(cluster_centers_indices)
n_clusters_

0

In [8]:
clusters = result['cluster'].unique()
labels = []
for i in range(len(clusters)):
    subset = result[result['cluster'] == clusters[i]]
    words = ' '.join([x for x in np.where(subset.all()!=0,subset.columns,None) if x and x!='text' and x!='cluster' and len(x.split()) == 1])
    labels.append(words)
labels_table = pd.DataFrame(zip(clusters,labels),columns=['cluster','label'])
result_labelled = pd.merge(result,labels_table,on='cluster',how='left')
result_labelled[['text', 'cluster', 'label']]

,text,cluster,label
0,nighti night this twitter thing is confus but ...,-1,
1,he he im good thanx ola cant wait until tomorr...,-1,
2,i wanna talk to you,-1,
3,im look forward to queen tomorrow too andi mur...,-1,
4,ew realli that full of fail,-1,
...,...,...,...
13995,gotta wake my bop up outta this lame ass tranc...,-1,
13996,why are you ok i am ill and i feel like i am g...,-1,
13997,you make evryth better i miss being your bestf...,-1,
13998,mom i hope your day was fill with love amp hap...,-1,


In [9]:
pd.pivot_table(result_labelled, index=['label'], values=['text'], aggfunc='count').sort_values(['text'], ascending=False).rename(columns={'text':'count of text'})

,count of text
label,
,14000


In [10]:
sil_score = silhouette_score(X, afprop.labels_)
sil_score

ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)